In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_validate
from scipy import optimize

from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import train_test_split

In [7]:
dataset_folder = "C:/Users/samsung/Desktop/Nouveau dossier/Etude de Cas Decisionnelle/data"
meta = pd.read_csv(f"{dataset_folder}/training_set_metadata.csv")
lc = pd.read_csv(f"{dataset_folder}/training_set.csv")

In [3]:
lc.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [4]:
targetTypes={6:'Microlensing', 15:'Explosive_V', 16:'Transits', 42:'Explosive_W', 52:'Explosive_X', 
                  53:'Long Periodic', 62:'Explosive_Y', 64:'Near Burst', 65:'Flare', 67:'Explosive_Z',
                  88:'AGN', 90:'SN_U', 92:'Periodic', 95:'SN_T'}
#targetTypes

In [8]:
groups = lc.groupby(['object_id', 'passband'])
times = groups.apply(lambda block: block['mjd'].values).reset_index().rename(columns={0: 'seq'})
flux = groups.apply(lambda block: block['flux'].values).reset_index().rename(columns={0: 'seq'})
err = groups.apply(lambda block: block['flux_err'].values).reset_index().rename(columns={0: 'seq'})
det = groups.apply(lambda block: block['detected'].astype(bool).values).reset_index().rename(columns={0: 'seq'})
times_list = times.groupby('object_id').apply(lambda x: x['seq'].tolist()).tolist()
flux_list = flux.groupby('object_id').apply(lambda x: x['seq'].tolist()).tolist()
err_list = err.groupby('object_id').apply(lambda x: x['seq'].tolist()).tolist()
det_list = det.groupby('object_id').apply(lambda x: x['seq'].tolist()).tolist()
object_list=times.groupby('object_id').apply(lambda x: x['object_id'].unique()[0]).tolist()
for t in sorted(meta['target'].unique()):
     obje= [t,meta[meta['target']== t]['target'].count(),targetTypes[t],meta[meta['target']== t]['hostgal_specz'].mean()]
     display(obje)

[6, 151, 'Microlensing', 0.0]

[15, 495, 'Explosive_V', 0.24357676767676767]

[16, 924, 'Transits', 0.0]

[42, 1193, 'Explosive_W', 0.22996244761106419]

[52, 183, 'Explosive_X', 0.22970601092896167]

...

[67, 208, 'Explosive_Z', 0.22544855769230773]

[88, 370, 'AGN', 0.9679970270270275]

[90, 2313, 'SN_U', 0.3630812797233031]

[92, 239, 'Periodic', 0.0]

[95, 175, 'SN_T', 1.217438285714286]

In [10]:
times.head()

,object_id,passband,seq
0,615,0,"[59819.1532, 59820.1047, 59821.1026, 59822.110..."
1,615,1,"[59750.4306, 59752.4147, 59767.3045, 59770.225..."
2,615,2,"[59750.4229, 59752.407, 59767.2968, 59770.2179..."
3,615,3,"[59750.4383, 59752.4224, 59767.3122, 59770.233..."
4,615,4,"[59750.445, 59752.4334, 59767.3233, 59770.2445..."


In [11]:
flux.head()

,object_id,passband,seq
0,615,0,"[6.878784, 39.364853000000004, -10.422381, -65..."
1,615,1,"[-816.4343259999999, -1061.4570310000001, -815..."
2,615,2,"[-544.810303, -681.858887, -548.01355, -554.90..."
3,615,3,"[-471.385529, -524.95459, -475.51605199999995,..."
4,615,4,"[-388.984985, -393.480225, -405.663818, -400.2..."


In [12]:
err.head()

,object_id,passband,seq
0,615,0,"[3.6331519999999995, 3.7756190000000003, 4.172..."
1,615,1,"[5.55337, 6.472994, 5.293019, 5.875329, 6.3068..."
2,615,2,"[3.6229519999999997, 4.0412040000000005, 3.462..."
3,615,3,"[3.8012129999999997, 3.552751, 3.340643, 3.736..."
4,615,4,"[11.395031, 3.5993459999999997, 3.496113, 3.83..."


In [9]:
det.head()

,object_id,passband,seq
0,615,0,"[False, True, False, True, True, True, True, T..."
1,615,1,"[True, True, True, True, True, True, True, Tru..."
2,615,2,"[True, True, True, True, True, True, True, Tru..."
3,615,3,"[True, True, True, True, True, True, True, Tru..."
4,615,4,"[True, True, True, True, True, True, True, Tru..."


In [13]:
lc1=lc[lc.object_id==615]
lc1.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [25]:
date1 = []
for row in lc.itertuples():
    tabl = row.mjd 
    tabl1=int(tabl)
    
    if tabl1 in date1:
        pass
    else:
        date1.append(tabl1) 
date1

[59750,
 59752,
 59767,
 59770,
 59779,
 59782,
 59797,
 59800,
 59807,
 59810,
 59813,
 59819,
 59820,
 59821,
 59822,
 59823,
 59835,
 59839,
 59842,
 59851,
 59854,
 59857,
 59864,
 59867,
 59870,
 59873,
 59874,
 59875,
 59876,
 59877,
 59878,
 59879,
 59880,
 59884,
 59887,
 60118,
 60124,
 60140,
 60141,
 60142,
 60143,
 60144,
 60145,
 60153,
 60162,
 60165,
 60168,
 60176,
 60177,
 60181,
 60184,
 60194,
 60197,
 60198,
 60199,
 60200,
 60201,
 60202,
 60206,
 60207,
 60208,
 60211,
 60221,
 60224,
 60227,
 60228,
 60229,
 60234,
 60237,
 60240,
 60249,
 60260,
 60261,
 60262,
 60263,
 60264,
 60490,
 60493,
 60499,
 60500,
 60501,
 60502,
 60508,
 60524,
 60525,
 60532,
 60535,
 60538,
 60546,
 60549,
 60554,
 60555,
 60556,
 60557,
 60558,
 60559,
 60560,
 60567,
 60574,
 60577,
 60580,
 60582,
 60583,
 60584,
 60585,
 60586,
 60587,
 60588,
 60593,
 60596,
 60603,
 60606,
 60609,
 60612,
 60613,
 60614,
 60615,
 60616,
 60617,
 60621,
 60624,
 59825,
 59896,
 59899,
 59902,


In [26]:
l=len(date1)
l


951

In [27]:
lobj=len(object_list)
lobj

7848

In [17]:
lc2=lc1.copy()
dateObjei=[]
passObjei=[]
y = lc2[["flux","flux_err","detected"]]  
for i in lc2.itertuples():
    tabli = i.mjd 
    tablj = i.passband 
    tabl1i=int(tabli)   
    dateObjei.append(tabl1i)
    passObjei.append(tablj)
X = pd.DataFrame(dateObjei,columns=['mjd'])
X2 = pd.DataFrame(dateObjei,columns=['passband'])  
X.loc[:,"passband"]=passObjei   
gaussian = GaussianProcessRegressor()
gaussian.fit(X,y)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=None,
             n_restarts_optimizer=0, normalize_y=False,
             optimizer='fmin_l_bfgs_b', random_state=None)

In [34]:
#df1=pd.Dataframe()
for j in date1:  
    for i in range(0,6):
        data1 = [[j, i]]
        X1 = pd.DataFrame(data1,columns=['mjd','passband'])
        y1 = gaussian.predict(X1)[0]
        data = [[615,j,i,y1[0],y1[1],y1[2]]]
        df = pd.DataFrame(data,columns=['object_id','mjd','passband','flux','flux_err','detected'])
        df1=df1.append(df) 
        
df1.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1.0
1,615,59750.4306,1,-816.434326,5.553370,1.0
2,615,59750.4383,3,-471.385529,3.801213,1.0
3,615,59750.4450,4,-388.984985,11.395031,1.0
4,615,59752.4070,2,-681.858887,4.041204,1.0


In [35]:
df1

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-5.448103e+02,3.622952e+00,1.000000e+00
1,615,59750.4306,1,-8.164343e+02,5.553370e+00,1.000000e+00
2,615,59750.4383,3,-4.713855e+02,3.801213e+00,1.000000e+00
3,615,59750.4450,4,-3.889850e+02,1.139503e+01,1.000000e+00
4,615,59752.4070,2,-6.818589e+02,4.041204e+00,1.000000e+00
5,615,59752.4147,1,-1.061457e+03,6.472994e+00,1.000000e+00
6,615,59752.4224,3,-5.249546e+02,3.552751e+00,1.000000e+00
7,615,59752.4334,4,-3.934802e+02,3.599346e+00,1.000000e+00
8,615,59752.4435,5,-3.558868e+02,1.042192e+01,1.000000e+00
9,615,59767.2968,2,-5.480136e+02,3.462291e+00,1.000000e+00
